## attempting spark

In [ ]:
# %pip install pyspark==3.3.1
# need to install java next

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 1.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 172.1 kB/s eta 0:00:0000:01
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845499 sha256=54272e21876125e31df4a9ab51a051822ce20de29308ac76e7999ddb56233447
  Stored in directory: /home/jovyan/.cache/pip/wheels/9c/aa/b1/8433fd8b1afe7eb31196cc74a42cd778bcb52636a428da079d
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [16]:
from pyspark.sql import SparkSession

In [18]:
data_location= "../../cephfs/data/glucose_records_2022-02-01.csv" #only reading 1 day of data
spark=SparkSession.builder.master("local"). \
                           appName('Resd_Glucose_Data'). \
                           getOrCreate()
pyspark_glucose_data=spark.read.csv(data_location, 
                                            header=True,
                                            sep=',')
# return pyspark_glucose_data

JAVA_HOME is not set


RuntimeError: Java gateway process exited before sending its port number

In [ ]:
spark = pyspark.sql.SparkSession\
    .builder \
    .appName(“GlucoseAnalysis) \
    .getOrCreate()
df = spark.read.csv(‘path_to_folder_with_all_csv’, header=True)

## just 1 csv in pandas

In [27]:
#didn't bother with versions
%pip install numpy==1.23.5 pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 5.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

In [2]:
'''initial read-in of 1 day of data, and saving-out a csv of just the cols we want'''
# usableCols = ["PatientId","GlucoseDisplayTime","Value"]
# df = pd.read_csv(data_location, low_memory=False, usecols=usableCols)
# df.to_csv("../cleanedData/2022-02-01 temp.csv", index=False)
day1 = pd.read_csv("../../cephfs/cleanedData/2022-02-01 temp.csv", parse_dates=['GlucoseDisplayTime'])
%time
day1

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.44 µs


,PatientId,GlucoseDisplayTime,Value
0,1Jxgxke6R3Uh2c9aRnMUF425ILTOg+t30nSUxUhiKqk=,2022-02-01 14:45:24.971,0
1,toBStbTTYI2GU28YdTryjNwSM/+HkkafNCkxar3xKD0=,2022-02-01 17:46:26.437,0
2,+XAhHhm+BkhqusxsZjnyQdS9CmRTmSvyJ16FyAoQbB4=,2022-02-01 14:58:52.229,0
3,+XAhHhm+BkhqusxsZjnyQdS9CmRTmSvyJ16FyAoQbB4=,2022-01-31 22:53:48.883,0
4,+XAhHhm+BkhqusxsZjnyQdS9CmRTmSvyJ16FyAoQbB4=,2022-01-31 22:38:44.655,0
...,...,...,...
1663020,VikQW3TwZOcHzKA7UFAIjJg7KfHAgb6h3heX6OQL3Qg=,2022-02-01 05:10:16.000,0
1663021,VikQW3TwZOcHzKA7UFAIjJg7KfHAgb6h3heX6OQL3Qg=,2022-02-01 08:15:16.000,0
1663022,VikQW3TwZOcHzKA7UFAIjJg7KfHAgb6h3heX6OQL3Qg=,2022-01-31 23:30:14.000,0
1663023,VikQW3TwZOcHzKA7UFAIjJg7KfHAgb6h3heX6OQL3Qg=,2022-01-31 23:35:15.000,0


In [3]:
day1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1663025 entries, 0 to 1663024
Data columns (total 3 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   PatientId           1663025 non-null  object        
 1   GlucoseDisplayTime  1663025 non-null  datetime64[ns]
 2   Value               1663025 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 38.1+ MB


In [4]:
'''bonus step: dropping any rows where the PatientId or timestamp is missing'''
initRows = day1.shape[0]
day1 = day1.dropna(subset=['PatientId','GlucoseDisplayTime'], how='any')
%time
print(initRows - day1.shape[0], "rows with missing patient ID or timestamp have been removed")

0 rows with missing patient ID or timestamp have been removed


### quick detour for datetime exploration

In [5]:
timestamp = day1['GlucoseDisplayTime'].loc[1]
timestamp

Timestamp('2022-02-01 17:46:26.437000')

In [6]:
print(timestamp.year)
print(timestamp.month)
print(timestamp.day)
print(timestamp.hour)
print(timestamp.minute)
print(timestamp.second)

2022
2
1
17
46
26


### back to EDA

In [7]:
'''round down to the minute so seconds = 0'''
day1['GlucoseDisplayTime'] = day1['GlucoseDisplayTime'].dt.floor('min') #'min' means 'minute' ('min' can be substituted with 'T' and 'Min' for literally the same effect
day1

,PatientId,GlucoseDisplayTime,Value
0,1Jxgxke6R3Uh2c9aRnMUF425ILTOg+t30nSUxUhiKqk=,2022-02-01 14:45:00,0
1,toBStbTTYI2GU28YdTryjNwSM/+HkkafNCkxar3xKD0=,2022-02-01 17:46:00,0
2,+XAhHhm+BkhqusxsZjnyQdS9CmRTmSvyJ16FyAoQbB4=,2022-02-01 14:58:00,0
3,+XAhHhm+BkhqusxsZjnyQdS9CmRTmSvyJ16FyAoQbB4=,2022-01-31 22:53:00,0
4,+XAhHhm+BkhqusxsZjnyQdS9CmRTmSvyJ16FyAoQbB4=,2022-01-31 22:38:00,0
...,...,...,...
1663020,VikQW3TwZOcHzKA7UFAIjJg7KfHAgb6h3heX6OQL3Qg=,2022-02-01 05:10:00,0
1663021,VikQW3TwZOcHzKA7UFAIjJg7KfHAgb6h3heX6OQL3Qg=,2022-02-01 08:15:00,0
1663022,VikQW3TwZOcHzKA7UFAIjJg7KfHAgb6h3heX6OQL3Qg=,2022-01-31 23:30:00,0
1663023,VikQW3TwZOcHzKA7UFAIjJg7KfHAgb6h3heX6OQL3Qg=,2022-01-31 23:35:00,0


next is to change PatientIDs to ints. this is for saving memory space and for our own sanity.

#### read in the cohort data

In [8]:
'''the index col wasn't removed before saving this csv, so repurpose it to the int ID column'''
cohort = pd.read_csv("../../cephfs/data/cohort.csv")
cohort = cohort.rename(columns={'Unnamed: 0':'intPatientId','UserId':'PatientId'})

'''check for duplicates'''
if len(cohort['intPatientId'].unique()) != len(cohort['PatientId'].unique()) \
    and len(cohort['PatientId'].unique()) != cohort.shape[0]:
    raise NameError("ERROR: duplicate patients found in cohort.csv file")

%time
cohort

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.2 µs


,intPatientId,PatientId,Gender,DOB,Age,DiabetesType,Treatment
0,0,5lZPrCk6qk8L6Jw+SU+NYDjP2u3hF6JNASttkRUi6Vc=,Female,1931-01-01,92,type-two,no
1,1,9qY9mZ+GV5Kd/O/NB1FuD4JqCUg2etZvhhFLwGTZfbY=,Male,1937-01-01,86,type-two,no
2,2,uhsyLhr4Zl6NfGbNBOXBvLo6ZD9OnFVF297de5/SrfE=,Female,1938-01-01,85,type-two,no
3,3,9uAVHBOgoCJ9hfcrLJvRxkwT/VpWZOFZ5iqAtRY3YH8=,Male,1938-01-01,85,type-two,no
4,4,Fyb156jU1edGykL7NCQBK+uTMpT7hNkq+/H1wwG1eeE=,Female,1939-01-01,84,type-two,no
...,...,...,...,...,...,...,...
7995,7995,y7iPpTOpOsK0Xx8V6oNYmyrhB5XclbVy7vNRugo2YH0=,Female,1992-01-01,31,type-two,yes-long-acting
7996,7996,KbLjey5zQXaIFNFgconzI5c9FJtIpzPR36YzWVICmpM=,Female,1993-01-01,30,type-two,yes-long-acting
7997,7997,JKatRtvvKlFbWRE7ewo4vaFM8c2HST13KhVKOZ7oz9U=,Female,1993-01-01,30,type-two,yes-long-acting
7998,7998,saW1uk+rYwjh90Jv26OS65cgJsY6LXs90C4tt0oGXWQ=,Female,1993-01-01,30,type-two,yes-long-acting


probably not worth saving this "cleaned" `cohort` dataframe, since i'm assuming it's more economical to just read in the data and do these two steps when we need it again

In [ ]:
#probably worth adding a function in here that just automatically checks for if any rows are missing data and then deal with that

In [9]:
'''make dict of all string IDs to int IDs'''
idDict = pd.Series(index=cohort['PatientId'], data=cohort['intPatientId'].values).to_dict()
idDict

{'5lZPrCk6qk8L6Jw+SU+NYDjP2u3hF6JNASttkRUi6Vc=': 0,
 '9qY9mZ+GV5Kd/O/NB1FuD4JqCUg2etZvhhFLwGTZfbY=': 1,
 'uhsyLhr4Zl6NfGbNBOXBvLo6ZD9OnFVF297de5/SrfE=': 2,
 '9uAVHBOgoCJ9hfcrLJvRxkwT/VpWZOFZ5iqAtRY3YH8=': 3,
 'Fyb156jU1edGykL7NCQBK+uTMpT7hNkq+/H1wwG1eeE=': 4,
 '86XfZ0fNI0VWOzWrlHcU/PgcZFypCu0ATO3zFqzSbtw=': 5,
 'JfJMH1qCpiYNuPOp/BAjx0DIWwpinBp0JfjSHjNw+c8=': 6,
 'EkW0PD80req7mL/5StxlnMlD/kIMJs6oGKHwKu9Vau8=': 7,
 'OyqSKorAj1OPZaevjbvObkf9HrTIzzoE2jOCBAk6lm0=': 8,
 '4n03YE5Q5c2Ge1O4+i4BsT4tlNGY6pCDa3+YEC1es+o=': 9,
 'uAMPYMNKPbpZUYFRaBMi4lvpis5a8oR0GOqZrYTQ8AM=': 10,
 'HN0cXF7eu32av2B7tHGLDNNWbVTFp6I8dLqVepLTuLs=': 11,
 'aTY0dn2ojwufciqoTqTwI3/CB2iYAra+XWAbJvUxknQ=': 12,
 'OMfjTKNcJTpWf6lvz8tPr4b4GjWTQiUyYy3n3/9+p1c=': 13,
 'Vns5tiWlfPr9ZAOIrgEz1dZv3BqP1p/WdbQGywiIkDg=': 14,
 '3yR+rNyx89hQ6BVD7duukxCJTYmJFxT6jwkXqsviHF4=': 15,
 'A0gx8UZPDrWZex0XH+EbKWzLz2zL6H+MtGJ72xpN4KM=': 16,
 'DPjfSVNYKu8gy19Pt2ciQt5tG1Hfv+c1RtMT/dH54ZM=': 17,
 'RCBM42DpSLdBF0MJx9MDkfsh6NC8HySZKt2M+fv7vKQ=': 18,
 'I

# replace string IDs with int IDs

next steps: replace 'PatientId' in `day1` dataframe using the key of {'UserId' : 'PatientIdInt'} from `cohort` dataframe
<br>
(not sure if a "merge + drop col + rename col" will be more economical, or if "replace" with dicts will be.)

#### with merge()

In [11]:
day1Merged = day1.merge(cohort[['PatientId','intPatientId']],
                         how = "inner",
                         on = "PatientId") \
        .drop(columns="PatientId") \
        .rename(columns={'intPatientId':'PatientId'})
day1Merged = day1Merged[['PatientId','GlucoseDisplayTime','Value']] #reorder columns for aesthetics
#maybe right merge? it'd eliminate any day1.PatientId's that don't appear in cohort.csv ?
%time
day1Merged

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 12.6 µs


,PatientId,GlucoseDisplayTime,Value
0,1560,2022-02-01 14:45:00,0
1,1560,2022-02-01 04:15:00,155
2,1560,2022-02-01 06:45:00,114
3,1560,2022-02-01 12:15:00,207
4,1560,2022-01-31 21:30:00,156
...,...,...,...
1663020,3735,2022-02-01 00:21:00,0
1663021,3735,2022-02-01 06:07:00,0
1663022,3735,2022-01-31 21:17:00,0
1663023,3735,2022-01-31 20:51:00,0


In [53]:
'''if day1 took about 8 seconds, round up to 10 seconds for the days with more data and the total time this will take for all 365 CSVs is about (in minutes):'''
10*365/60

60.833333333333336

granted, my 8 sec run was because of like 7.9 seconds of wall time. CPU was miniscule. also the above might be more optimizable (ask katie about this since merge() is SQL based)

#### with replace()

In [54]:
'''this took more than a minute so i didn't even let it finish'''
day1Replaced = day1.replace({'PatientId': idDict})
%time
day1Replaced


KeyboardInterrupt



In [55]:
'''this ALSO took more than a minute so cell killed'''
day1Replaced = day1.replace(idDict)
%time
day1Replaced

KeyboardInterrupt: 

In [56]:
#executing before the next cell so %time will only measure the function i want to measure
day1Replaced = day1.copy()

In [ ]:
'''THIS KILLED THE KERNEL LMAO'''
day1Replaced['PatientId'] = day1Replaced['PatientId'].replace(idDict)
%time
day1Replaced

great news running this answered the question for me.

merge it is.